## ニューラルネットワーク入門

In [ ]:
- ニューラルネットワークは、あるニューロンの出力が別のニューロンの入力になり刺激が伝搬していくようなイメージ(脳の神経モデルの話)
- 各ニューロンの計算を複数接続することで複雑な計算をすることが出来る
- これを数学的なモデルに落とし込んだものがパーセプトロン
- n個の入力に対してそれぞれ対応したn個の重みと固定値の入力バイアス(b)がある

#### 単純パーセプトロン

In [2]:
#  単純パーセプトロンのコード

import math

def f(z):
    return 1.0 / (1.0 + math.exp(-z)) # Sigmoid

def perceptron(x):
    w = [-1.64, -0.98, 1.31]
    b = -0.05
    
    z= b+ x[0] * w[0] + x[1] * w[1] + x[2] * w[2]
    output = f(z)
    
    return output

x = [0.2, 0.3, -0.1]
print(perceptron(x))
x= [-0.2, -0.1, 0.9]
print(perceptron(x))

0.3093841557917043
0.8256347143825868


In [6]:
# Numpy使って簡略化

import numpy as np

def f(z):
    return 1.0 / (1.0 + np.exp(-z)) # Sigmoid

def perceptron(x):
    w = np.array([-1.64, -0.98, 1.31])
    b = -0.05
    
    # 内積をnp.dotで計算してくれる
    z = b + np.dot(x, w)
    output = f(z)
    
    return output

x = np.array([0.2, 0.3, -0.1])
print(perceptron(x))
x= np.array([-0.2, -0.1, 0.9])
print(perceptron(x))



0.3093841557917043
0.8256347143825868


- 同じ結果が出る
- np.dotめちゃ便利
- sigmoidの出力は0-1の間の結果を返すので、2クラス分類のタスクにも応用出来る
- 単純パーセプトロンを識別器とした場合は、重み(w,b)の最適値を求めることが学習になる

#### 多層パーセプトロン

- 単純パーセプトロンの出力を別の単純パーセプトロンの入力にすることで多数のパーセプトロンをつなげる構造をとるのが多層パーセプトロン
- 多層パーセプトロンで同列の並びが単純パーセプトロンがレイヤ
- レイヤの中の単純パーセプトロンのことをユニットという
- 単純パーセプトロンが10個ならんだレイヤ= ユニット数10のレイヤ
- 入力ベクトル　入力層
- 最後の出力 出力層、最終層
- 間が中間層とか隠れ層(昔たかパイと島田さんが隠れ層の話をしていたことを思い出した)
- 最終層を1ユニットにすることで出力値をsigmoidにして2クラス識別器的に使うことが出来る

In [23]:
# 多層パーセプトロンのコードサンプル numpy

import numpy as np

def f_1(z):
    return np.maximum(z, 0) #Relu

def layer_1(x):
    # それぞれの入力に対する重み
    W_1 = np.array([
    [-0.423, -0.795, 1.223],
    [1.402, 0.885, -1.701],
    ])
    # 固定値の入力バイアス
    b_1 = np.array([0.546, 0.774])
    
    z_1 = b_1 + np.dot(W_1, x)
    output_1 = f_1(z_1)
    
    return output_1

x = np.array([0.2, 0.4, -0.1])
out_1 = layer_1(x)
print(out_1)

[0.0211 1.5785]


In [24]:
# 次の層

def f_2(z):
    return 1.0 / (1.0 + np.exp(-z)) # Sigmoid

def layer_2(x):
    # それぞれの入力に対する重み
    W_2 = np.array([
    [1.567, -1.645],
    ])
    # 固定値の入力バイアス
    b_2 = np.array([ 0.255])
    
    z_2 = b_2 + np.dot(W_2, x)
    output_2 = f_2(z_2)
    return output_2

out_2 = layer_2(out_1)
print(out_2)


[0.09041578]


## Karas

- Karasはニューラルネットワークの操作を抽象化したライブラリ
- バックエンドではTensorFlowがデフォルトで使われている

In [27]:
# Karasによる多層パーセプトロンの実装

from keras.layers import Dense
from keras.models import Sequential

model = Sequential()

# Denseクラスで層を表現
model.add(Dense(units=2, activation='relu', input_dim=3))
model.add(Dense(units=1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam')


Instructions for updating:
Colocations handled automatically by placer.


- Karasでは角層の重みは乱数で初期化される
  - 初期値にはいくつかのパターンがある！！
- 具体的な重みは隠蔽、抽象化されて、層の単位でコードを書く
- 重みにアクセスすることは可能

In [28]:
# 重みにアクセスする
print(model.layers[0].get_weights())

[array([[0.558535  , 0.32071495],
       [0.09770143, 1.0626979 ],
       [0.57016635, 0.02764893]], dtype=float32), array([0., 0.], dtype=float32)]


#### 活性化関数(activation function)

- activateで表現される
- パーセプトロンでは、全ての入力に重みが掛け合わされてそれらをすべて足して、その結果に対して変換をかける(サンプロコードのf関数)
- この変換をかける関数のことを活性化関数という

メジャーな関数
- Relu まずはこれだけ押さえればOK 精度や収束しやすさで優れた性能を備える 近年ではほとんどReluかその発展形が使われる
- Sigmoid 2クラス識別器として最終層に使われる 出力が0-1の間に収束されるので出力値を確立のようにみなせる
- Hyperbolic tangent(tanh) Sigmoidと同じ用途で使われるが、出力が-1-1なので範囲が広い分sigmoidより高性能 

Sigmoidとtanhは最終層に使われる

Reluは a<=0 のときは0
a>0のときはその値を返すが、なぜそれがいいかはわかっていない。


#### 損失関数

- loss引数で指定するもの
- 識別器の出力と学習データがどのような意味で近いことが望ましいかを定義するのが損失関数
- 損失関数が定義する望ましさに向かって学習するので重要だが、パターンがある程度決まっており解いている問題次第で自ずと決まってくるもの

#### 最適化手法(optimizer)
- 多層パーセプトロンは損失関数によって定義された望ましさに向かってパラメータを調整していく、これがパラメータの最適化
- その最適化の手法をoptimizerで定義する

Adamを使う


#### 学習 エポックとバッチサイズ
- fit関数にはepochsとbatch_sizeというパラーメータがある
- 学習時に繰り返し処理で徐々に重みを変えることでじわじわと最適化
- この学習時のパラメータがエポックとバッチサイズ

- この流れを繰り返す
　　- 1. xのうちbatch_size個をパーセプトロンに入力し、最終出力(予測結果)を得る
  - 2. 予測結果と学習データを比較し、損失関数によって損失を求める
  - 3. 損失を小さくするように重みを更新
  
 - 全ての学習データを全件使い切ると1エポック
 - バッチサイズは1回の処理でどれくらいのデータを使うかを定義
 
 エポック128
 バッチも1024

#### 学習率
- 繰り返しでじわじわと重みを最適化していくが、一回の重みの更新でどの程度の重みの値を増減させるか決めるパラメータが学習率
- 大きいと1回あたりの変化大きいという意味
- ベストプラクティスは大きな学習率で大雑把に重みを最適値に近づけてその後学習率を小さくして最適値を目指す

0.05,0.01とかがデフォルト値のことが多い

- 重みを流用するってイメージなのかな??あんまし具体的にイメージ出来てない

逆伝搬するときの変化率

# ニューラルネットワークによる識別器

In [ ]:
import numpy as numpy
from keras.layers import Dense
from keras.models import Sequential

model = Sequential()
model.add(Dense(32, input_dim=100, activation='relu'))
model.add(Dense(10, activation='softmax'))
mode.compile(loss='categorical_crossentropy', optimizer='adam')

X = np.array([
    []
])

- one-hot encoding出てきた　これはsoftmaxとイコールかな
- 最終層ユニット数が2以上の多層パーセプトロンを使った多クラス識別にはsoftmaxを使う。こんな特徴がある。
  - 出力が0と1の間に収まるので、その次元が1でそれ以外は0という設定にマッチ(one-hot encoding)
  - 活性化関数を適用した層のすべての出力の和が1 (単純に1 or 0で出るって話??ちょっとよくわからないので聞く)
  - Softmaxを適用した層の各ユニットの出力値の、大きい値と小さい値の差が開く(Softmaxを通すと大小の比が大きくなる)
- 明確に0か1ではなく、0か1に使い値という扱いになるから、出力が最も大きい場合にそのクラスみたいな判定として使う
- 「i番目のユニットの出力が1でそれ以外が0なら、クラスID=iと予測する」場合には、損失関数にcategorical_crossentropyを使う

In [3]:
# kerasでone-hot表現に変換　便利ツール

import numpy as np
from keras.utils import to_categorical

y = np.array([0,1,2,3,4])
y_one_hot = to_categorical(y)

print(y_one_hot)

Using TensorFlow backend.
/Users/shnagai/.pyenv/versions/anaconda3-5.3.1/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/Users/shnagai/.pyenv/versions/anaconda3-5.3.1/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/Users/shnagai/.pyenv/versions/anaconda3-5.3.1/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16",

[[1. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0.]
 [0. 0. 1. 0. 0.]
 [0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 1.]]


- sparse_categorical_crossentropyが、one-hotのパーセプトロンの出力と非one-hot表現の教師データの橋渡しをするってのがよくわかってないぞ

### 09.2対話エージェントへの適用

In [ ]:
import unicodedata
from os.path import dirname, join, normpath

import Mecab
import neologdn
import numpy as np
import pandas as pd
from keras.layers import Dense
from keras.models import Sequential
from keras.utils import to_categorical
from sklearn.feature_extraction.text import TfidfVectorizer

Class DialogueAgent:
    def 